# Web scraper

In this notebook, I am going to scrap all the data from movie critic website.\
There are datasets from rottentomatoes and imdb.

## 1. IMDb

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [11]:
url = "https://www.imdb.com/chart/moviemeter/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
    "X-Amzn-Trace-Id": "Root=1-65e35243-4893ea3e5f2895705cbb6c1f"
}
page = requests.get(url, headers=headers, timeout=5)
page

<Response [200]>

In [33]:
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <script>
   window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1709383694285);
        }
    })
  </script>
  <title>
   Most Popular Movies
  </title>
  <meta content="As determined by IMDb users" data-id="main" name="description"/>
  <meta content="IMDb" property="og:site_name"/>
  <meta content="Most Popular Movies" property="og:title"/>
  <meta

In [38]:
# scrap and parse all the needed data
urls_name = []
for tag in soup.find_all("a"):
    if tag.get("href").startswith("/title/"):
        href = tag.get("href")
        words = href.split("/")
        href = "/title/" + words[2]
    else:
        continue

    full_arialabel = tag.get("aria-label")
    if full_arialabel and full_arialabel.startswith("View title"):
        title = full_arialabel.split("page for ")[1]
    else:
        title = None

    if href and title:
        urls_name.append((href, title))

csv = "urls_name.csv"
df = pd.DataFrame(urls_name, columns=["url", "name"])
df.to_csv(csv, index=False)

### Go through all the urls to get reviews
Use Microsoft Edge Driver and selenium to press "load more" button on every website of movies.

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver_path = r"C:\Users\yuuna\WebDriver\chromedriver-win64\chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0")
driver = webdriver.Chrome(executable_path=driver_path, options=options)

In [14]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:1080'
os.environ['https_proxy'] = 'https://127.0.0.1:1080'

In [34]:
import time
# read csv file and open all urls to scrap the needed data
csv = "urls_name.csv"
df = pd.read_csv(csv)
reviews = []

for url, name in zip(df["url"], df["name"]):
    url = "https://www.imdb.com" + url + "/reviews"
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    temp_header = soup.find("div", class_="header")
    review_num = temp_header.find("span", class_=None)
    review_num = review_num.text.split(" ")[0]
    review_num = int(review_num.replace(",", ""))

    if(review_num > 25):
        load_more_button = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, "ipl-load-more__button")))

        while load_more_button.is_displayed():
            load_more_button.click()
            # Add some delay to let the "Load More" button appear again
            time.sleep(3)
            driver.implicitly_wait(5)
    
    source = driver.page_source

    soup = BeautifulSoup(source, "html.parser")
    
    count = 0
    for review in soup.find_all("div", class_="review-container"):
        score = review.find("span", class_=None)
        if score:
            score = score.text
        else:
            score = None
        rev_title = review.find("a", class_="title")
        rev_text = review.find("div", class_="text show-more__control")
        if score and rev_title and rev_text:
            reviews.append((name, score, rev_title.text, rev_text.text))
            count += 1

    print(name, "has", count, "reviews.")

    driver.quit()


蜘蛛夫人 has 362 reviews.
邊緣禁地 has 0 reviews.
奧本海默 has 2276 reviews.
沙丘：第二部 has 386 reviews.
可憐的東西 has 392 reviews.
愛愛愛上你 has 175 reviews.
夢想集中營 has 128 reviews.
沙丘 has 4473 reviews.
雷鬼之父：音樂無國界 has 99 reviews.
天眼救援 has 57 reviews.
死侍與金鋼狼 has 0 reviews.
Saltburn has 654 reviews.
The Iron Claw has 136 reviews.
機密特務：阿蓋爾 has 298 reviews.
真愛挑日子 has 221 reviews.
滯留生 has 239 reviews.
蜂刑者 has 371 reviews.
親愛的陌生人 has 127 reviews.
恐怖感恩劫 has 233 reviews.
師情化慾 has 34 reviews.
墜惡真相 has 204 reviews.
升等之旅 has 50 reviews.
Drive-Away Dolls has 47 reviews.
Mean Girls has 134 reviews.
This Is Me... Now 這就是現在的我 has 135 reviews.
花月殺手 has 729 reviews.
Players has 39 reviews.
美式小說 has 94 reviews.
旺卡 has 356 reviews.
驚奇隊長2 has 636 reviews.
Avgrunden has 30 reviews.
不可能的任務：致命清算 第一章 has 860 reviews.
之前的我們 has 249 reviews.
Barbie芭比 has 1106 reviews.
哥吉拉與金剛：新帝國 has 0 reviews.
愛爾蘭之願 has 0 reviews.
Lisa Frankenstein has 54 reviews.
絕地盟約 has 188 reviews.
No Way Up has 32 reviews.
One Life has 31 reviews.
Boy Kills World

In [35]:
imdb_csv = "imdb_reviews.csv"
df = pd.DataFrame(reviews, columns=["movie", "score", "title", "review"])
df.to_csv(imdb_csv, index=False)

Now we have 47742 raw reviews from imdb. Let's clean it a little bit for the correct shapes of score.\
\
Next, I want to convert the score into sentiments(positive or negative).\
This convertion will take the definition of "freshness" on rottentomatoes.com:\
\
When at least 60% of reviews for a movie or TV show are positive, a red tomato is displayed to indicate its Fresh status.\
When less than 60% of reviews for a movie or TV show are positive, a green splat is displayed to indicate its Rotten status.\
\
I assume the score these reviews gave (?/10) is the positive level of their review.\
If the score >=6, the review will be marked as fresh(positive).\
If the score <6, the review will be marked as rotten(negative).

In [47]:
# add one column to imdb_reviews.csv to indicate the sentiment
import pandas as pd

df = pd.read_csv("imdb_reviews.csv")
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df.dropna(subset=["score"])

df['sentiment'] = df['score'].apply(lambda x: 1 if int(x) >= 6 else 0)
df.to_csv("imdb_reviews.csv", index=False)

In [48]:
df = pd.read_csv("imdb_reviews.csv")
positive_count = df[df["sentiment"] == 1].shape[0]
negative_count = df[df["sentiment"] == 0].shape[0]
print("Total reviews:", df.shape[0])
print("Positive reviews:", positive_count)
print("Negative reviews:", negative_count)

Total reviews: 46299
Positive reviews: 30362
Negative reviews: 15937


In [27]:
import pandas as pd
import re

df = pd.read_csv("ttest_imdb_reviews.csv")
# remove all \n in review and title
# df["review"] = df["review"].str.replace("\n", " ")
# df["title"] = df["title"].str.replace("\n", " ")
# df["review"] = df["review"].apply(lambda x: re.sub("\n", " ", x))
# df["title"] = df["title"].apply(lambda x: re.sub("\n", " ", x))
df.to_csv("test_imdb_reviews.csv", index=False)

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import re
import string
from nltk.corpus import stopwords

stopwords = set(stopwords.words("english"))

translator = str.maketrans("", "", string.punctuation)

def clean_text(text):
    # final_text = []
    text = text.translate(translator)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stopwords.words("english")]
    text = ''.join([i for i in text if not i.isdigit()])
    return text

def filter_word(text):
    text = text.split()
    filtered_words = [word for word in text if word not in stopwords]
    return " ".join(filtered_words)

df = pd.read_csv("test_imdb_reviews.csv")
# clean the review and title
# for data in df:
df["review"] = df["review"].apply(lambda x: filter_word(x))
df["title"] = df["title"].apply(lambda x: filter_word(x))
# print(df["review"][0])
# df["title"][0]
df.to_csv("test_imdb_reviews.csv", index=False, mode="w")

In [3]:
import pandas as pd

df = pd.read_csv("train_imdb_reviews.csv")
positive_count = df[df["sentiment"] == 1].shape[0]
negative_count = df[df["sentiment"] == 0].shape[0]
print("Total reviews:", df.shape[0])
print("Positive reviews:", positive_count)
print("Negative reviews:", negative_count)
print("ratio:", positive_count/negative_count)

Total reviews: 41669
Positive reviews: 27295
Negative reviews: 14374
ratio: 1.8989147071100598


In [4]:
df = pd.read_csv("test_imdb_reviews.csv")
positive_count = df[df["sentiment"] == 1].shape[0]
negative_count = df[df["sentiment"] == 0].shape[0]
print("Total reviews:", df.shape[0])
print("Positive reviews:", positive_count)
print("Negative reviews:", negative_count)
print("ratio:", positive_count/negative_count)

Total reviews: 4630
Positive reviews: 3067
Negative reviews: 1563
ratio: 1.962252079334613


## RottenTomatoes

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd